In [6]:
%matplotlib inline

import random
import numpy as np
from deap import base, creator, tools, algorithms, benchmarks
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
import pandas as pd
import time



In [7]:
def eaSimple(pop, toolbox, cxpb, mutpb, ngen, stats, hof):
    pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb, mutpb, ngen, stats=stats, halloffame=hof, verbose=False)
    
    return pop, stats, hof, logbook

def plot_evolution(logbook):
    gen = logbook.select("gen")
    min_vals = logbook.select("min")

    plt.plot(gen, min_vals)
    plt.xlabel('Generation')
    plt.ylabel('Minimum Fitness Value')
    plt.title('Minimum Fitness Value over Generations')
    plt.grid(True)
    plt.show()


In [8]:
def sharing(distance, sigma, alpha):
    res = 0
    if distance<sigma:
        res += 1 - (distance/sigma)**alpha
    return res

def shared_fitness(individual, population, sigma, alpha, fitness):
    num = fitness(individual)[0]

    dists = distance_matrix([individual], population)[0]
    tmp = [sharing(d, sigma, alpha) for d in dists]
    den = sum(tmp)

    if den == 0:
        den = 1
    
    return num/den,

In [9]:
# [... (population_size, ngen, is_shared_fitness) ...]
hyper_param = [(50, 500, False), (50, 1000, False), (50, 5000, False), (50, 10000, False),
               (50, 500, True), (50, 1000, True), (50, 5000, True), (50, 10000, True),
               (100, 500, False), (100, 1000, False), (100, 5000, False), (100, 10000, False),
               (100, 500, True), (100, 1000, True), (100, 5000, True), (100, 10000, True),]
# [... (benchmark_function, low, high) ...]
benchmark_functions = [(benchmarks.ackley, -15, 30), 
                       (benchmarks.rosenbrock, -5.12, 5.12),
                       (benchmarks.rastrigin, -5.12, 5.12),
                       (benchmarks.bohachevsky, -100, 100),
                       (benchmarks.schaffer, -100, 100)]

experiment_data = []

for benchmark_function, low, high in benchmark_functions:
    for population_size, ngen, is_shared_fitness in hyper_param:

        print(f"function: {benchmark_function.__name__}, population_size: {population_size}, ngen: {ngen}, is_shared_fitness: {is_shared_fitness}")

        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        toolbox = base.Toolbox()

        toolbox.register("attr_float", random.uniform, low, high)
        toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        toolbox.register("select", tools.selTournament, tournsize=3)
        toolbox.register("mate", tools.cxBlend, alpha=0.5)
        toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.5, indpb=0.1)

        pop = toolbox.population(n=population_size)
        if is_shared_fitness:
            toolbox.register('evaluate', shared_fitness, population=pop, sigma=0.1, alpha=1., fitness=benchmark_function)
        else:
            toolbox.register("evaluate", benchmark_function)
        
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min)
        stats.register("avg", np.mean)


        hof = tools.HallOfFame(1)
        cxpb, mutpb = 0.5, 0.075

        start_time = time.time()
        pop, stats, hof, logbook = eaSimple(pop, toolbox, cxpb, mutpb, ngen, stats, hof)
        end_time = time.time()

        elapsed_time = end_time - start_time

        experiment_data.append([benchmark_function.__name__, is_shared_fitness, population_size, ngen, benchmark_function(hof[0]), str(hof[0]), elapsed_time])

output_frame = pd.DataFrame(experiment_data, columns=["benchmark_func", "is_shared_fitness", "population_size", "ngen", "best_fitness", "best_individual", "time (s)"])
output_frame.to_csv("fitness_sharing_study.csv", index=False)

function: ackley, population_size: 50, ngen: 500, is_shared_fitness: False
function: ackley, population_size: 50, ngen: 1000, is_shared_fitness: False
function: ackley, population_size: 50, ngen: 5000, is_shared_fitness: False
function: ackley, population_size: 50, ngen: 10000, is_shared_fitness: False
function: ackley, population_size: 50, ngen: 500, is_shared_fitness: True
function: ackley, population_size: 50, ngen: 1000, is_shared_fitness: True
function: ackley, population_size: 50, ngen: 5000, is_shared_fitness: True
function: ackley, population_size: 50, ngen: 10000, is_shared_fitness: True
function: ackley, population_size: 100, ngen: 500, is_shared_fitness: False
function: ackley, population_size: 100, ngen: 1000, is_shared_fitness: False
function: ackley, population_size: 100, ngen: 5000, is_shared_fitness: False
function: ackley, population_size: 100, ngen: 10000, is_shared_fitness: False
function: ackley, population_size: 100, ngen: 500, is_shared_fitness: True
function: ack